In [46]:
def hex_to_binary(hex_string):
    binary_array = []
    
    for hex_digit in hex_string:
        binary_digit = bin(int(hex_digit, 16))[2:]
        padded_binary_digit = binary_digit.zfill(4)
        binary_array.append(padded_binary_digit)
    
    return binary_array

def subs(input_nibble):
    s_box = {
    '0000': '1010',
    '0001': '0000',
    '0010': '1001',
    '0011': '1110',
    '0100': '0110',
    '0101': '0011',
    '0110': '1111',
    '0111': '0101',
    '1000': '0001',
    '1001': '1101',
    '1010': '1100',
    '1011': '0111',
    '1100': '1011',
    '1101': '0100',
    '1110': '0010',
    '1111': '1000'
    }
    return s_box[input_nibble]

def sub_nibbles(input_values):
    output_values = []
    for i in input_values:
        output_values.append(subs(i))
    return output_values

def generate_round_keys(master_key):
    round_keys = []

    w0, w1, w2, w3 = int(master_key[0], 2), int(master_key[1], 2), int(master_key[2], 2), int(master_key[3], 2)

    rc1 = int('1110', 2)
    rc2 = int('1010', 2)

    w4 = w0 ^ int(subs(format(w3, '04b')), 2) ^ rc1
    w5 = w1 ^ w4
    w6 = w2 ^ w5
    w7 = w3 ^ w6

    w8 = w4 ^ int(subs(format(w7, '04b')), 2) ^ rc2
    w9 = w5 ^ w8
    w10 = w6 ^ w9
    w11 = w7 ^ w10

    K1 = [format(w4, '04b'), format(w5, '04b'), format(w6, '04b'), format(w7, '04b')]
    K2 = [format(w8, '04b'), format(w9, '04b'), format(w10, '04b'), format(w11, '04b')]

    round_keys.append(K1)
    round_keys.append(K2)

    return round_keys

def xor_binary_numbers(num1, num2):
    if len(num1) != 4 or len(num2) != 4:
        raise ValueError("Input binary numbers must be 4 bits long")

    int_num1 = int(num1, 2)
    int_num2 = int(num2, 2)

    result = int_num1 ^ int_num2

    result_binary = format(result, '04b')

    return result_binary

def add_round_key(list1, list2):
    result = []
    for i, j in zip(list1, list2):
        result.append(xor_binary_numbers(i, j))
    return result
        
def mix_columns(input_block):
    constant_matrix = [
        [1, 4],
        [4, 1]
    ]

    c0, c1 = int(input_block[0], 2), int(input_block[1], 2)
    c2, c3 = int(input_block[2], 2), int(input_block[3], 2)

    def multiply_in_finite_field(a, b):
        m = 0
        while b > 0:
            if b & 1:
                m ^= a
            a <<= 1
            if a & 0x10:
                a ^= 0x13
            b >>= 1
        return m & 0x0F

    d0 = multiply_in_finite_field(constant_matrix[0][0], c0) ^ multiply_in_finite_field(constant_matrix[0][1], c1)
    d1 = multiply_in_finite_field(constant_matrix[1][0], c0) ^ multiply_in_finite_field(constant_matrix[1][1], c1)

    d2 = multiply_in_finite_field(constant_matrix[0][0], c2) ^ multiply_in_finite_field(constant_matrix[0][1], c3)
    d3 = multiply_in_finite_field(constant_matrix[1][0], c2) ^ multiply_in_finite_field(constant_matrix[1][1], c3)

    d0_binary = format(d0, '04b')
    d1_binary = format(d1, '04b')
    d2_binary = format(d2, '04b')
    d3_binary = format(d3, '04b')

    return [d0_binary, d1_binary, d2_binary, d3_binary]

def shift_row(lst):
    if len(lst) != 4:
        return lst
    return [lst[2], lst[1], lst[0], lst[3]]

def nibbles_to_hex(nibbles):
    binary_str = ''.join(nibbles)
    hex_value = hex(int(binary_str, 2))[2:].upper().zfill(4)
    return hex_value

def inv_mix_columns(input_block):
    constant_matrix_inverse = [
        [9, 2],
        [2, 9]
    ]

    c0, c1 = int(input_block[0], 2), int(input_block[1], 2)
    c2, c3 = int(input_block[2], 2), int(input_block[3], 2)

    def multiply_in_finite_field(a, b):
        m = 0
        while b > 0:
            if b & 1:
                m ^= a
            a <<= 1
            if a & 0x10:
                a ^= 0x13
            b >>= 1
        return m & 0x0F

    d0 = multiply_in_finite_field(constant_matrix_inverse[0][0], c0) ^ multiply_in_finite_field(constant_matrix_inverse[0][1], c1)
    d1 = multiply_in_finite_field(constant_matrix_inverse[1][0], c0) ^ multiply_in_finite_field(constant_matrix_inverse[1][1], c1)

    d2 = multiply_in_finite_field(constant_matrix_inverse[0][0], c2) ^ multiply_in_finite_field(constant_matrix_inverse[0][1], c3)
    d3 = multiply_in_finite_field(constant_matrix_inverse[1][0], c2) ^ multiply_in_finite_field(constant_matrix_inverse[1][1], c3)

    d0_binary = format(d0, '04b')
    d1_binary = format(d1, '04b')
    d2_binary = format(d2, '04b')
    d3_binary = format(d3, '04b')

    return [d0_binary, d1_binary, d2_binary, d3_binary]

def inv_subs(input_nibble):
    inv_s_box = {
    '1010': '0000',
    '0000': '0001',
    '1001': '0010',
    '1110': '0011',
    '0110': '0100',
    '0011': '0101',
    '1111': '0110',
    '0101': '0111',
    '0001': '1000',
    '1101': '1001',
    '1100': '1010',
    '0111': '1011',
    '1011': '1100',
    '0100': '1101',
    '0010': '1110',
    '1000': '1111'
    }
    return inv_s_box[input_nibble]

def inv_sub_nibbles(input_values):
    output_values = []
    for i in input_values:
        output_values.append(inv_subs(i))
    return output_values  

def encryption(plaintext,master_key):
    print("Original Inputs:\nPlain-text:",plaintext,"\nMaster-key:",master_key)
    state = hex_to_binary(plaintext)
    master_key = hex_to_binary(master_key)
    print("Separated Nibbles:\nPlain-text:",state,"\nMaster-key:",master_key)
    state = sub_nibbles(state)
    print("Sub-Nibbles:\nPlain-text:",nibbles_to_hex(state))
    round_keys = generate_round_keys(master_key)
    print("Generate Round Keys:\nRound Key 1:",nibbles_to_hex(round_keys[0]),"\nRound Key 2:",nibbles_to_hex(round_keys[1]))
    state = add_round_key(state,round_keys[0])
    print("Add Round Key 1:",nibbles_to_hex(state))
    state = mix_columns(state)
    print("Mix Columns:",nibbles_to_hex(state))
    state = shift_row(state)
    print("Shift Row:",nibbles_to_hex(state))
    state = sub_nibbles(state)
    print("Sub-Nibbles:",nibbles_to_hex(state))
    state = add_round_key(state, round_keys[1])
    print("Add Round Key 2:",nibbles_to_hex(state))
    state = shift_row(state)
    print("Shift Row:",nibbles_to_hex(state))
    ciphertext = nibbles_to_hex(state)
    print("Ciphertext:",ciphertext)  
    
def decryption(ciphertext, masterkey):
    round_keys = generate_round_keys(hex_to_binary(masterkey))
    print("Ciphertext:",ciphertext)
    print("Round Keys:",nibbles_to_hex(round_keys[0]),nibbles_to_hex(round_keys[1]))
    state = shift_row(hex_to_binary(ciphertext))
    print("Shift Row:",nibbles_to_hex(state))
    state = add_round_key(state,round_keys[1])
    print("Add round key 2:",nibbles_to_hex(state))
    state = inv_sub_nibbles(state)
    print("Inverse Sub-Nibbles:",nibbles_to_hex(state))
    state = shift_row(state)
    print("Shift Row:",nibbles_to_hex(state))
    state = inv_mix_columns(state)
    print("Inverse Mix Columns:",nibbles_to_hex(state))
    state = add_round_key(state,round_keys[0])
    print("Add round key 1:",nibbles_to_hex(state))
    state = inv_sub_nibbles(state)
    print("Inverse Sub-Nibbles:",nibbles_to_hex(state))
    plaintext = nibbles_to_hex(state)
    print("Decryptyed Plain-text:",plaintext)
    
def decrypt_text_block(ciphertext_block, masterkey):
    decrypted_block = []
    ciphertext_block = ciphertext_block.split()
    round_keys = generate_round_keys(hex_to_binary(masterkey))
    for ciphertext in ciphertext_block:
        round_keys = generate_round_keys(hex_to_binary(masterkey))
        state = shift_row(hex_to_binary(ciphertext))
        state = add_round_key(state,round_keys[1])
        state = inv_sub_nibbles(state)
        state = shift_row(state)
        state = inv_mix_columns(state)
        state = add_round_key(state,round_keys[0])
        state = inv_sub_nibbles(state)
        plaintext = nibbles_to_hex(state)
        decrypted_block.append(plaintext)
    return decrypted_block

In [50]:
print("------------------Encryption Pocket AES------------------")

encryption("903b", "02cc")

------------------Encryption Pocket AES------------------
Original Inputs:
Plain-text: 903b 
Master-key: 02cc
Separated Nibbles:
Plain-text: ['1001', '0000', '0011', '1011'] 
Master-key: ['0000', '0010', '1100', '1100']
Sub-Nibbles:
Plain-text: DAE7
Generate Round Keys:
Round Key 1: 57B7 
Round Key 2: AD61
Add Round Key 1: 8D50
Mix Columns: 9B57
Shift Row: 5B97
Sub-Nibbles: 37D5
Add Round Key 2: 9AB4
Shift Row: BA94
Ciphertext: BA94


In [51]:
print("------------------Decryption Pocket AES------------------")

decryption("ba94", "02cc")

------------------Decryption Pocket AES------------------
Ciphertext: ba94
Round Keys: 57B7 AD61
Shift Row: 9AB4
Add round key 2: 37D5
Inverse Sub-Nibbles: 5B97
Shift Row: 9B57
Inverse Mix Columns: 8D50
Add round key 1: DAE7
Inverse Sub-Nibbles: 903B
Decryptyed Plain-text: 903B


In [49]:
print("------------------Test D1------------------")

plaintext = "903b"
print("Plaintext:",plaintext)
state = hex_to_binary(plaintext)
print("Sub Nibbles:",nibbles_to_hex(sub_nibbles(state)))
print("ShiftRow:",nibbles_to_hex(shift_row(state)))
print("MixColumns:",nibbles_to_hex(mix_columns(state)))
masterkey = "02cc"
print("Masterkey:",masterkey)
masterkey = hex_to_binary(masterkey)
rk = generate_round_keys(masterkey)
print("Round Keys:",nibbles_to_hex(rk[0]),nibbles_to_hex(rk[1]))

print("------------------Test D2------------------")

decryption("F3D7","40ee")

print("------------------Test D3------------------")

try:
    with open('secret.txt', 'r') as encrypted_file:
        encrypted_data = encrypted_file.read().splitlines()
        print("Reading from encrypted file secret.txt...")
except FileNotFoundError:
    print("Error: 'secret.txt' not found.")

key = "149c"

decrypted_text = ""

for ciphertext_block in encrypted_data:
    decrypted_block = decrypt_text_block(ciphertext_block, key)
    
    decrypted_content = ''
    for i in decrypted_block:
        decrypted_content += chr(int(i[:2], 16))
        if int(i[2:], 16) != 0:
            decrypted_content += chr(int(i[2:], 16))
            
    print('\nDecrypted Result')
    print("--------------------")
    print(decrypted_content)
    print("--------------------")
    
with open('plain.txt', 'w') as output_file:
    output_file.write(decrypted_text)

print("Decryption completed. Decrypted text saved to 'plain.txt'.")

------------------Test D1------------------
Plaintext: 903b
Sub Nibbles: DAE7
ShiftRow: 309B
MixColumns: 9297
Masterkey: 02cc
Round Keys: 57B7 AD61
------------------Test D2------------------
Ciphertext: F3D7
Round Keys: 8868 3BD5
Shift Row: D3F7
Add round key 2: E822
Inverse Sub-Nibbles: 3FEE
Shift Row: EF3E
Inverse Mix Columns: A171
Add round key 1: 2919
Inverse Sub-Nibbles: E282
Decryptyed Plain-text: E282
------------------Test D3------------------
Reading from encrypted file secret.txt...

Decrypted Result
--------------------
Gentlemen, you can't fight in here. This is the war room.
--------------------
Decryption completed. Decrypted text saved to 'plain.txt'.
